In [3]:
import pandas as pd
anime_df=pd.read_csv(r"C:\Users\Avanish chandra\Downloads\data bases\archive (2)\anime.csv")
rating_df=pd.read_csv(r"C:\Users\Avanish chandra\Downloads\data bases\archive (2)\anime.csv")

In [4]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
rating_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
!pip install pandas sqlalchemy psycopg2 -q


In [7]:

import random
from datetime import datetime, timedelta


In [9]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Define DB credentials (make sure these are correct)
db_user = 'postgres'
db_password = 'Likith1612'
db_host = 'localhost'
db_port = '5432'
db_name = 'anime_db'

# 2. Create connection engine
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")






In [10]:
from sqlalchemy import text

sql = """
DROP TABLE IF EXISTS anime CASCADE;

CREATE TABLE anime (
    anime_id     INT PRIMARY KEY,
    anime_title  TEXT,
    genre        TEXT,
    anime_type   TEXT,
    episodes     INT,
    avg_rating   FLOAT,
    members      INT
);
"""

with engine.connect() as conn:
    conn.execute(text(sql))

print("✅ Recreated 'anime' table.")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [13]:
anime_df.rename(columns={
    'name': 'anime_title',
    'type': 'anime_type',
    'rating': 'avg_rating'
}, inplace=True)
print(anime_df.columns.tolist())
# Should return:
# ['anime_id', 'anime_title', 'genre', 'anime_type', 'episodes', 'avg_rating', 'members']


['anime_id', 'anime_title', 'genre', 'anime_type', 'episodes', 'avg_rating', 'members']


In [14]:
anime_df.to_sql("anime", con=engine, index=False, if_exists='append')
print(f"✅ Loaded {len(anime_df)} anime rows.")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
ratings_df = rating_df[rating_df['rating'] != -1].copy()
ratings_df['time_id'] = None  # add time_id if applicable

ratings_df.to_sql("ratings", con=engine, index=False, if_exists='replace')
print(f"✅ Loaded ratings: {len(ratings_df)} rows (excluding -1)")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [16]:
# Step 1: Explode genres from anime_df
anime_genres = anime_df[['anime_id', 'genre']].dropna()
anime_genres['genre'] = anime_genres['genre'].str.split(',')
anime_genres = anime_genres.explode('genre')
anime_genres['genre'] = anime_genres['genre'].str.strip()

# Step 2: Create genre dimension table
genre_df = anime_genres['genre'].drop_duplicates().reset_index(drop=True).to_frame()
genre_df['genre_id'] = genre_df.index + 1
genre_df = genre_df[['genre_id', 'genre']].rename(columns={'genre': 'genre_name'})

# ✅ Save to database
genre_df.to_sql("genre2", con=engine, index=False, if_exists='replace')
print(f"✅ Loaded {len(genre_df)} genres.")

# Step 3: Build bridge table with correct genre_id mapping
anime_genre_bridge_df = anime_genres.merge(
    genre_df, left_on='genre', right_on='genre_name'
)[['anime_id', 'genre_id']]

# ✅ Save to database
anime_genre_bridge_df.to_sql("anime_genre_bridge2", con=engine, index=False, if_exists='replace')
print(f"✅ Loaded {len(anime_genre_bridge_df)} anime-genre bridge records.")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [17]:
from sqlalchemy import text

# Drop the bridge table that depends on genre
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS anime_genre_br1 CASCADE;"))
    print("✅ Dropped 'anime_genre_br1' table (FK dependency).")

# Now safely drop genre table
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS genre CASCADE;"))
    print("✅ Dropped 'genre' table.")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [18]:
fact_df = rating_df[rating_df['rating'] > 0][['user_id', 'anime_id', 'rating']].copy()

# ✅ Step 2: Load into fact_anime_ratings
# Since rating_id is SERIAL (auto), we do NOT include it in the insert
fact_df.to_sql("fact_anime_ratings", con=engine, index=False, if_exists='append')

print(f"✅ Inserted {len(fact_df)} rows into fact_anime_ratings.")

KeyError: "['user_id'] not in index"